# Семинар 3. Исправление опечаток

In [3]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
punct = set(punctuation)
from sklearn.metrics import classification_report

Возьмем данные с соревнования Dialog Evaluation 2015 по исправлению опечаток. Данные представляют собой набор предложений (правильное - ошибочное). Задача найти слова с ошибками и заменить их на правильный вариант.

Недостатком тут является то, что не всегда можно правильно сопоставить слова правильного предложения и ошибочного (из-за слов с пропущенным или добавленным пробелом). Из статьи авторов корпуса не очень понятно, как они решали эту проблему, поэтому я просто удалил все такие предложения.

In [6]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [3]:
# Посмотрим на пары предложений
i = 2
print(bad[i])
print(true[i])

Пояним эту мысль.
Поясним эту мысль


In [4]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [5]:
pprint(align_words(true[1], bad[1]))

[('апофеозом', 'опофеозом'),
 ('дня', 'дня'),
 ('для', 'для'),
 ('меня', 'меня'),
 ('сегодня', 'сегодня'),
 ('стала', 'стала'),
 ('фраза', 'фраза'),
 ('услышанная', 'услышанная'),
 ('в', 'в'),
 ('новостях', 'новостях')]


Вытащим только неправильные варианты и заодно посчитаем процент ошибок.

In [6]:
mistakes = []
total = 0

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        total += 1

In [7]:
print('Доля ошибок -', len(mistakes) / total)

Доля ошибок - 0.13034358769476628


In [8]:
mistakes[:5]

[('симпатичнейшее', 'симпатичнейшое'),
 ('апофеозом', 'опофеозом'),
 ('поясним', 'пояним'),
 ('получатся', 'полчатся'),
 ('очень', 'оччччень')]

Обернем в Counter, чтобы сразу увидеть частотные ошибки.

In [9]:
Counter(mistakes).most_common(10)

[(('сегодня', 'седня'), 24),
 (('вообще', 'вобще'), 18),
 (('вообще', 'ваще'), 17),
 (('естественно', 'естесственно'), 17),
 (('хочется', 'хочеться'), 16),
 (('кстати', 'кстате'), 16),
 (('очень', 'ооочень'), 14),
 (('как-то', 'както'), 9),
 (('очень', 'оооочень'), 9),
 (('это', 'ето'), 9)]

Из-за того, что процент ошибок довольно низкий, не очень выгодно будет находить исправление для каждого слова. Нужен какой-то более простой классификатор, который выделит ошибочные слова, чтобы потом только их и редактировать.

Самый простой способ это сделать - составить словарь правильных слов и потом сравнивать с ним. Чтобы не делать этого вручную, можно взять какой-нибудь корпус текстов, прошедщих редактуру. Новостные тексты для этого хорошо подходят.

Я заранее собрал небольшой корпус новостных текстов и немного почистил его удалив отдельную пунктуацию и пунктуацию на границах слов.

In [10]:
corpus = open('corpus_ng.txt', encoding='utf8').read().splitlines()

In [11]:
# нормализация нам тут не нужна так как нужно находить слова в разных формах
print(corpus[1].split()[:10])

['судя', 'по', 'всему', 'русская', 'православная', 'церковь', 'нашла', 'долгожданную', 'национальную', 'идею']


## Задание 1.
Напишите функцию, которая будет предсказывать ошибочные слова на основе корпуса.

In [12]:
# создайте множество, чтобы проверять вхождения
vocab = set(elem for article in corpus for elem in article.split())

In [13]:
def predict_mistaken(word, vocab):
    '''
    :: input: word, vocabulary
    :: output: 1 or 0
    '''
    return 1 - int(word in vocab)

In [14]:
# для оценки создайте два списка y_true и y_pred
# пройдитесь по предложениям
# сопоставите слова с помощью функции align_words
# пройдитесь по парам слов и
# если слова одинаковые добавьте в y_true 0
# если слова разные добавьте в y_true 1
# предскажите ошибочность слова из bad списка
# добавьте предсказание в список y_pred

y_true = []
y_pred = []

for i in range(len(true)):
    for pair in align_words(true[i], bad[i]):
        if pair[0] != pair[1]:
            y_true.append(1)
        else:
            y_true.append(0)
        y_pred.append(predict_mistaken(pair[1], vocab))

In [15]:
# оцените качество с помощью classification_report
print(classification_report(y_true, y_pred))

             precision    recall  f1-score   support

          0       0.98      0.89      0.93      8707
          1       0.55      0.88      0.68      1305

avg / total       0.92      0.89      0.90     10012



### Генерация исправлений

Теперь нужно думать о том, как исправить неправильные слова. Посмотрим как это можно делать на примере известной реализации Питера Норвига.

Основная идея - сделать словарь правильных слов (у нас уже есть), расчитать вероятность каждого слова в корпусе.

In [4]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [17]:
WORDS.most_common(10)

[('в', 67679),
 ('и', 55933),
 ('на', 27860),
 ('не', 21627),
 ('что', 18299),
 ('с', 18224),
 ('по', 13117),
 ('а', 9696),
 ('как', 8958),
 ('к', 8907)]

In [18]:
# фунцкия расчета вероятности слова
N = sum(WORDS.values())
def P(word, N=N):
    "Вычисляем вероятность слова"
    return WORDS[word] / N

In [19]:
P('апофеоз')

1.8025455548325345e-06

Чтобы найти исправления нужно сгенерировать возможные исправления и выбрать те, которые есть в словаре. Если есть несколько вариантов, то выбрать тот, у котогоро наибольшая вероятность.

In [20]:
def correction(word):
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

def candidates(word):
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in WORDS)

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) >= 2]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [21]:
%%time
correction('опефеоз')

Wall time: 143 ms


'апофеоз'

Давайте поподробнее разберем, что происходит в функции edits.

In [22]:
word = 'опефеоз'
splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]

In [23]:
splits[:10]

[('', 'опефеоз'),
 ('о', 'пефеоз'),
 ('оп', 'ефеоз'),
 ('опе', 'феоз'),
 ('опеф', 'еоз'),
 ('опефе', 'оз'),
 ('опефео', 'з'),
 ('опефеоз', '')]

In [24]:
deletes = [L + R[1:] for L, R in splits if R]

In [25]:
deletes[:10]

['пефеоз', 'оефеоз', 'опфеоз', 'опееоз', 'опефоз', 'опефез', 'опефео']

In [26]:
transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]

In [27]:
transposes[:10]

['поефеоз', 'оепфеоз', 'опфееоз', 'опеефоз', 'опефоез', 'опефезо']

In [28]:
letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
replaces = [L + c + R[1:] for L, R in splits if R for c in letters]

In [29]:
len(replaces)

231

In [30]:
inserts = [L + c + R for L, R in splits for c in letters]

In [31]:
len(inserts)

264

In [32]:
inserts[:10]

['йопефеоз',
 'цопефеоз',
 'уопефеоз',
 'копефеоз',
 'еопефеоз',
 'нопефеоз',
 'гопефеоз',
 'шопефеоз',
 'щопефеоз',
 'зопефеоз']

Для оценки используем просто долю правильных исправлений.

In [33]:
# До этого мы уже считали долю ошибок во всех предложениях.
# Поэтому если ничего не менять то доля правильных исправлений уже будет 100 - 13 = 87 %.
# Наш подход соответственно должен показывать лучший результат.
correct = 0
total = 0

for i in range(len(true)):
    for pair in align_words(true[i], bad[i]):
        predicted = correction(pair[1])
        if predicted == pair[0]:
            correct += 1
        total += 1
    if not i % 10:
        print(i)
        print(correct / total)

0
0.7333333333333333
10
0.8384615384615385
20
0.8441558441558441
30
0.8434343434343434
40
0.847953216374269
50
0.8518518518518519
60
0.8458333333333333
70
0.8502475247524752
80
0.8501594048884166
90
0.8526522593320236
100
0.8512544802867383
110
0.8547439126784215
120
0.8546017014694509
130
0.8534798534798534
140
0.8523581681476419
150
0.8580481622306717
160
0.8573113207547169
170
0.8568220101066817
180
0.8541666666666666
190
0.8552104208416834
200
0.8581048581048581
210
0.8593466424682396
220
0.8598971722365039
230
0.859958932238193
240
0.8639350599149594
250
0.8619170033051781
260
0.8628209637706648
270
0.8603786342123056
280
0.8606001936108422
290
0.8584758942457231
300
0.859375
310
0.8601173020527859
320
0.8605046781967678
330
0.8591549295774648
340
0.8582974137931034
350
0.8576288929599581
360
0.8575772934617334
370
0.8583478045150087
380
0.8595898673100121
390
0.8602558029369967
400
0.8619809788912085
410
0.8613615870153292
420
0.8611910560106265
430
0.8613861386138614
440
0.86255

In [34]:
print(correct / total)

0.8598681582101478


In [35]:
%%time
correction('нав')

Wall time: 0 ns


'на'

In [36]:
%%time
correction('насмехатьсяаававттававаываываы')

Wall time: 2.85 s


'насмехатьсяаававттававаываываы'

Посмотрим какие исправления выбираются для самых частотных опечаток.

In [37]:
[(wt[0], wt[1], correction(wt[1])) for wt, _ in Counter(mistakes).most_common(10)]

[('сегодня', 'седня', 'сеня'),
 ('вообще', 'вобще', 'вообще'),
 ('вообще', 'ваще', 'ваще'),
 ('естественно', 'естесственно', 'естественно'),
 ('хочется', 'хочеться', 'хочется'),
 ('кстати', 'кстате', 'кстати'),
 ('очень', 'ооочень', 'очень'),
 ('как-то', 'както', 'какао'),
 ('очень', 'оооочень', 'оооочень'),
 ('это', 'ето', 'ето')]

### Метрики близости слов.

Вместо того, чтобы генерировать все варианты, можно искать похожие слова в словаре. Для этого нужно задать метрику похожести. Для исправления опечаток часто используются расстояния редактирования (количество редактирований, которые нужно сделать в строке a, чтобы прийти к b.

In [40]:
# в питоне есть библиотека для нахождения близких строк

In [8]:
from difflib import get_close_matches

In [42]:
%%time
get_close_matches('опофеоз', WORDS.keys(), n=1)

Wall time: 698 ms


['апофеоз']

Работает тоже не очень быстро.

Недавно вышла библиотека, в которой реализованы многие методы нахождения расстояний.

In [9]:
import textdistance

In [21]:
def get_closest_match_with_metric(text, lookup, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word)

    return similarities.most_common(1)[0]

In [45]:
%%time
get_closest_match_with_metric('опофиоз', WORDS, textdistance.hamming)

Wall time: 3.07 s


('апофеоз', 0.7142857142857143)

In [46]:
%%time
get_closest_match_with_metric('апофиоз', WORDS, textdistance.levenshtein)

Wall time: 33.3 s


('апофеоз', 0.8571428571428572)

Ждать так долго мы не можем, поэтому попробуем что-то побыстрее.

Многие вещи, которые медленно решаются в питоне, можно оптимизировать с помощью матричных и векторных операций.

Сделаем поиск похожих по векторам символов, из которых состоит слово.

In [10]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [48]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [49]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [17]:
def get_closest_match_vec(text, X, vec, TOPN=3):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    return [id2word[top] for top in topn]

In [51]:
%%time
get_closest_match_vec('опофеоз', X, vec)

Wall time: 64 ms


['апофеоз', 'апофеозом', 'апофеоза']

## Задание 2. 


Напишите функцию, которая принимает слово и находит ближайшее к нему в словаре (сгенерируйте несколько кандидатов с помощью get_closest_match_vec, а затем посчитайте метрики близости до кадого слова и выведете самое близкое). 

In [20]:
def get_closest_hybrid_match(text, X, vec, metric=textdistance.levenshtein):
    variants = get_closest_match_vec(text, X, vec, 7)
    closest = get_closest_match_with_metric(text, variants, metric)[0]
    return closest

In [53]:
get_closest_hybrid_match('алкогнль', X, vec)

'алкоголь'

In [54]:
# оцените качество также как и раньше (если будет долго работать возьмите кусок данных)
# посмотрите на ошибки
correct = 0
total = 0

incorrect = [] # сюда будем сохранять ошибки

for i in range(len(true)):
    for pair in align_words(true[i], bad[i]):
        predicted = get_closest_hybrid_match(pair[1], X, vec)
        if predicted == pair[0]:
            correct += 1
        else:
            incorrect.append((pair[0], pair[1], predicted))
        total += 1
    if not i % 10:
        print(i)
        print(correct / total)

0
0.5333333333333333
10
0.8076923076923077
20
0.8095238095238095
30
0.8106060606060606
40
0.8128654970760234
50
0.8180354267310789
60
0.8097222222222222
70
0.8131188118811881
80
0.8172157279489904
90
0.8182711198428291
100
0.8145161290322581
110
0.8178001679261125
120
0.8205723124516628
130
0.819047619047619
140
0.8154477101845523
150
0.8238276299112801
160
0.8225235849056604
170
0.8231330713082537
180
0.8205128205128205
190
0.8221442885771543
200
0.8249158249158249
210
0.8271324863883848
220
0.8277634961439588
230
0.8287474332648871
240
0.8333977580208736
250
0.8303341902313625
260
0.8308125219838199
270
0.8289384719405003
280
0.8305905130687319
290
0.8283048211508554
300
0.8299278846153846
310
0.8302052785923754
320
0.8296002268216615
330
0.8287765810549572
340
0.8278556034482759
350
0.8272703480764197
360
0.8274201723264065
370
0.8285785164971471
380
0.8291917973462002
390
0.8297015632401705
400
0.8306657388077012
410
0.8311541929666366
420
0.8313039628071729
430
0.8314679294016358


In [55]:
# Качество хуже, чем 87%, и даже хуже, чем результат функции correction, т.е. функция get_closest_hybrid_match исправляет в том числе и корректные слова и делает это чаще, чем функция correction.
# Возможно, чтобы качество действительно выросло, нужно брать большее значение TOPN, но скорее всего дело в том, что correction включает в качестве возможного варианта в том числе и себя, а также умеет совершать 2 операции и в определённой степени учитывает контекст (рекурсивный спуск только на 2 действия, т.е. большая часть слова останется неизменной), но минус его работы в том, что он очень долго работает.
print(correct / total)

0.8304035157810628


In [15]:
import pandas as pd

In [56]:
labels = "correct_word false_word predicted_word".split()
df = pd.DataFrame.from_records(incorrect, columns=labels)

In [57]:
df

,correct_word,false_word,predicted_word
0,симпатичнейшее,симпатичнейшое,пластичнейшими
1,шпионское,шпионское,шпионские
2,гламурный,гламурный,лагерный
3,бонда,бонда,банда
4,superheadz,superheadz,super
5,clap,clap,place
6,camera,camera,america
7,получатся,полчатся,ополчатся
8,язычки,язычки,язычка
9,очень,оччччень,чечни


In [58]:
# как видно, проблема в том, что в словаре отсутствуют многие корректные слова, так что даже теоретически невозможно предсказать это слово
for word in "симпатичнейшее шпионское гламурный бонда".split():
    print('get_closest_hybrid_match', word + ':', get_closest_hybrid_match(word, X, vec))
    print('correction', word + ':', correction(word))
    print("%s in vocabulary: %d" % (word, word in vocab))

get_closest_hybrid_match симпатичнейшее: пластичнейшими
correction симпатичнейшее: симпатичнейшее
симпатичнейшее in vocabulary: 0
get_closest_hybrid_match шпионское: шпионские
correction шпионское: шпионской
шпионское in vocabulary: 0
get_closest_hybrid_match гламурный: лагерный
correction гламурный: гламурный
гламурный in vocabulary: 0
get_closest_hybrid_match бонда: банда
correction бонда: фонда
бонда in vocabulary: 0


### Добавим в наш словарь все слова из массива true, чтобы их в принципе можно было бы предсказать и обучим заново

In [59]:
vocab = list(WORDS.keys()) + [word for sent in true for word in sent.lower().split()]
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [60]:
for word in "симпатичнейшее шпионское гламурный бонда".split():
    print('get_closest_hybrid_match', word + ':', get_closest_hybrid_match(word, X, vec))
    print('correction', word + ':', correction(word))
    print("%s in vocabulary: %d" % (word, word in vocab))

get_closest_hybrid_match симпатичнейшее: симпатичнейшее
correction симпатичнейшее: симпатичнейшее
симпатичнейшее in vocabulary: 1
get_closest_hybrid_match шпионское: шпионское
correction шпионское: шпионской
шпионское in vocabulary: 1
get_closest_hybrid_match гламурный: гламурный
correction гламурный: гламурный
гламурный in vocabulary: 1
get_closest_hybrid_match бонда: бонда
correction бонда: фонда
бонда in vocabulary: 1


In [61]:
correct = 0
total = 0

incorrect = [] # сюда будем сохранять ошибки

for i in range(len(true)):
    for pair in align_words(true[i], bad[i]):
        predicted = get_closest_hybrid_match(pair[1], X, vec)
        if predicted == pair[0]:
            correct += 1
        else:
            incorrect.append((pair[0], pair[1], predicted))
        total += 1
    if not i % 10:
        print(i)
        print(correct / total)

0
1.0
10
0.9153846153846154
20
0.9177489177489178
30
0.9267676767676768
40
0.9298245614035088
50
0.9355877616747182
60
0.9375
70
0.9368811881188119
80
0.9394261424017003
90
0.93713163064833
100
0.9327956989247311
110
0.9345088161209067
120
0.9334880123743233
130
0.9347985347985348
140
0.9343814080656186
150
0.9366286438529785
160
0.9363207547169812
170
0.9359910162829871
180
0.9326923076923077
190
0.935370741482966
200
0.936988936988937
210
0.9382940108892922
220
0.9391602399314481
230
0.940041067761807
240
0.9424043293390028
250
0.9430774880646345
260
0.9433696799155822
270
0.9435429344151454
280
0.9428848015488868
290
0.9421461897356143
300
0.9429086538461539
310
0.9425219941348973
320
0.9415934221718174
330
0.9409003037834852
340
0.9399245689655172
350
0.9398063334205705
360
0.9401926001013685
370
0.9409575787645745
380
0.9408926417370326
390
0.9410232117479868
400
0.9410809556947344
410
0.9413886384129847
420
0.9417755147221607
430
0.9427464485578992
440
0.9424825544512582
450
0.94

In [62]:
# Как видно, качество выросло почти на 11%
print(correct / total)

0.9390731122652817


In [63]:
df = pd.DataFrame.from_records(incorrect, columns=labels)
df

,correct_word,false_word,predicted_word
0,получатся,полчатся,ополчатся
1,очень,оччччень,чечни
2,насчет,нащщот,защищено
3,в,вобщем,общем
4,общем,как,как
5,как,вы,вы
6,вы,знаете,знаете
7,знаете,из,из
8,из,моего,моего
9,моего,не,не


### Как видно, довольно большой корпус здорово помог бы улучшить качество, хотя лично я не вижу ничего плохого в том, чтобы включать правильные слова, опечатки которых мы тестируем, учитывая наш векторный метод (вместе с метрикой Левенштейна).
### Из 3-4 строк таблицы выше видно, что есть ещё одна проблема: есть некоторые сочетания слов, которые иногда люди пишут слитно (или наоборот раздельно, когда надо слитно: см. 9-10 строки), что, естественно, ломает дальнейшую проверку, т.е. алгоритм, может, предсказывает и верно, но в качестве корректного рассматривается предыдущее или следующее слово. Это относится также к словам через дефис. Попробуем усовершенствовать функцию align_words из предположения, что нам нужно будет сделать не больше 1 действия (это, вероятно, добавит FP, но, думаю, немного), рассчитывая расстояние левенштейна от объединения или разъединения двух слов до корректного, а также что не будет слитных написаний 3 отдельных слов.

In [12]:
def align_words(sent_1, sent_2):
    hyper_parameter = 2
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()

    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    first = 0
    second = 0
    tokens_1_copy = []
    tokens_2_copy = []
    while first < len(tokens_1) and second < len(tokens_2):
        if tokens_1[first] == tokens_2[second]:
            tokens_1_copy.append(tokens_1[first])
            tokens_2_copy.append(tokens_2[second])
        elif first + 1 < len(tokens_1) and \
                textdistance.levenshtein(tokens_1[first] + tokens_1[first + 1], tokens_2[second]) < hyper_parameter:
            tokens_1_copy.append(tokens_1[first] + ' ' + tokens_1[first + 1])
            tokens_2_copy.append(tokens_2[second])
            first += 1
        elif second + 1 < len(tokens_2) and \
                textdistance.levenshtein(tokens_1[first], tokens_2[second] + tokens_2[second + 1]) < hyper_parameter:
            tokens_1_copy.append(tokens_1[first])
            tokens_2_copy.append(tokens_2[second] + ' ' + tokens_2[second + 1])
            second += 1
        else:
            tokens_1_copy.append(tokens_1[first])
            tokens_2_copy.append(tokens_2[second])
        first += 1
        second += 1
    return list(zip(tokens_1_copy, tokens_2_copy))

### Эта фнукция не совсем честная, поскольку использует правильные ответы, т.е. в реальных условиях не поможет, но более честно вычислить качество в данной задаче, безусловно, она поможет. Исправить это можно, если завести, например, словарь биграмм. Тогда можно будет ссылаться не на существующие ответы, а на него. Хотя всё равно остаётся проблема случаев типа "не_давнего", поскольку встречаются оба варианта, но в этом случае можно ориентироваться на частотность соответствующих биграммы и униграммы.

In [91]:
align_words("в общем как вы знаете из моего недавнего прошлого", "вобщем как вы знаете из моего не давнего прошлого")

[('в общем', 'вобщем'),
 ('как', 'как'),
 ('вы', 'вы'),
 ('знаете', 'знаете'),
 ('из', 'из'),
 ('моего', 'моего'),
 ('недавнего', 'не давнего'),
 ('прошлого', 'прошлого')]

In [92]:
i = 0
pprint(align_words(true[i], bad[i]))
pprint(align_words(true[i + 12], bad[i + 12]))

[('симпатичнейшее', 'симпатичнейшое'),
 ('шпионское', 'шпионское'),
 ('устройство', 'устройство'),
 ('такой', 'такой'),
 ('себе', 'себе'),
 ('гламурный', 'гламурный'),
 ('фотоаппарат', 'фотоаппарат'),
 ('девушки', 'девушки'),
 ('бонда', 'бонда'),
 ('миниатюрная', 'миниатюрная'),
 ('модель', 'модель'),
 ('камеры', 'камеры'),
 ('superheadz', 'superheadz'),
 ('clap', 'clap'),
 ('camera', 'camera')]
[('хорошо', 'хороше'),
 ('что', 'что'),
 ('на', 'на'),
 ('выходных', 'выходгых'),
 ('не', 'не'),
 ('было', 'было'),
 ('стен', 'стен'),
 ('только', 'только'),
 ('деревья', 'деревья'),
 ('да', 'да'),
 ('ручьи', 'ручьи')]


In [99]:
# Также поменяем в соответствии с функцией align_words добавляемые слова из массива true
vocab = list(WORDS.keys()) + [words[0] for i in range(len(true)) for words in align_words(true[i], bad[i])]
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1, 1))
X = vec.fit_transform(vocab)

In [100]:
correct = 0
total = 0

incorrect = [] # сюда будем сохранять ошибки

for i in range(len(true)):
    for pair in align_words(true[i], bad[i]):
        predicted = get_closest_hybrid_match(pair[1], X, vec)
        if predicted == pair[0]:
            correct += 1
        else:
            incorrect.append((pair[0], pair[1], predicted))
        total += 1
    if not i % 10:
        print(i)
        print(correct / total)

0
1.0
10
0.9612403100775194
20
0.9434782608695652
30
0.9417721518987342
40
0.94140625
50
0.9451612903225807
60
0.9457579972183588
70
0.9442379182156134
80
0.9457446808510638
90
0.9429695181907571
100
0.9381165919282511
110
0.9394957983193277
120
0.9410852713178295
130
0.9419970631424376
140
0.9410958904109589
150
0.9428571428571428
160
0.9421145894861194
170
0.9415073115860517
180
0.9411134903640257
190
0.9427710843373494
200
0.9440963855421687
210
0.945
220
0.9454935622317596
230
0.9461127108185932
240
0.9481223383662408
250
0.9485104817947775
260
0.9485734413525889
270
0.9485443466486121
280
0.9489334195216548
290
0.9479750778816199
300
0.9485404754739694
310
0.9485898942420682
320
0.9474581084919057
330
0.9466113416320885
340
0.945748987854251
350
0.945464079706345
360
0.945925361766946
370
0.9465705765407555
380
0.9465925567907202
390
0.9466192170818505
400
0.9465613382899628
410
0.9467148340483179
420
0.9467731204258151
430
0.9473911168607159
440
0.947045117559839
450
0.9463689482

In [101]:
# "Честное" качество больше почти на 0.7-0.8%.
print(correct / total)

0.9463731865932966


In [102]:
df = pd.DataFrame.from_records(incorrect, columns=labels)
df

,correct_word,false_word,predicted_word
0,получатся,полчатся,ополчатся
1,очень,оччччень,чечни
2,насчет,нащщот,защищено
3,в общем,вобщем,общем
4,недавнего,не давнего,не знаю
5,хорошо,хороше,хорошее
6,потому,патаму,депутатам
7,что,шта,штат
8,повтыкав,поффтыкав,кофты
9,что-то,чтото,что


In [113]:
# Качество улучшилось, но, как ни странно, вектора у "не давнего" и "недавнего" довольно сильно отличаются, возможно, из-за разных контекстов: "не давнего, а ..." и "недавного времени/..."
get_closest_match_vec('не давнего', X, vec, 20).index('недавнего')

17

### Попробуем ещё улучшить качество, рассматривая не односимволные n-граммы, а, допустим, от 1-грамм до 3-грамм, чтобы получить чуть лучшее векторное представление слова, соответственно, слова, соответствующие близким словам, должны стать ещё ближе, а также, собственно, учитывать контекст в слове (до последовательностей из 3 букв).

In [18]:
vocab = list(WORDS.keys()) + [words[0] for i in range(len(true)) for words in align_words(true[i], bad[i])]
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1, 3))
X = vec.fit_transform(vocab)

In [22]:
correct = 0
total = 0

incorrect = [] # сюда будем сохранять ошибки

for i in range(len(true)):
    for pair in align_words(true[i], bad[i]):
        predicted = get_closest_hybrid_match(pair[1], X, vec)
        if predicted == pair[0]:
            correct += 1
        else:
            incorrect.append((pair[0], pair[1], predicted))
        total += 1
    if not i % 10:
        print(i)
        print(correct / total)

0
1.0
10
0.9534883720930233
20
0.9434782608695652
30
0.9392405063291139
40
0.939453125
50
0.9419354838709677
60
0.9415855354659249
70
0.9417596034696406
80
0.9446808510638298
90
0.9419862340216323
100
0.9390134529147982
110
0.9394957983193277
120
0.9410852713178295
130
0.9412628487518355
140
0.9417808219178082
150
0.9422222222222222
160
0.9421145894861194
170
0.9409448818897638
180
0.9421841541755889
190
0.9442771084337349
200
0.944578313253012
210
0.945
220
0.9450643776824035
230
0.9457013574660633
240
0.9473480449090205
250
0.9485104817947775
260
0.9489256780556534
270
0.9485443466486121
280
0.9492566257272139
290
0.9489096573208723
300
0.9494432741498646
310
0.949177438307873
320
0.9491621698381142
330
0.9482710926694329
340
0.9479082321187584
350
0.9470372312532774
360
0.947956334094948
370
0.9485586481113321
380
0.9480425326244563
390
0.9482799525504152
400
0.9484200743494424
410
0.9482953262587491
420
0.9483255710800621
430
0.9486847779215178
440
0.9485278542681636
450
0.94824707

In [23]:
# качество выросло на 3 сотых и почти достигло 0.95
print(correct / total)

0.9496748374187094


In [25]:
# довольно естественные ошибки, хотя кое-где и довольно некорректно.
labels = "correct_word false_word predicted_word".split()
df = pd.DataFrame.from_records(incorrect, columns=labels)
df

,correct_word,false_word,predicted_word
0,поясним,пояним,стояния
1,получатся,полчатся,ополчатся
2,насчет,нащщот,нащокина
3,основная,основая,основа
4,в общем,вобщем,общем
5,ящика,ящека,щекам
6,хорошо,хороше,хорошем
7,потому,патаму,датам
8,что,шта,штат
9,что-то,чтото,тото


### Не вижу смысла добавлять mystem, чтобы исправление было такой же частью речи, поскольку тогда мы будем ориентироваться на работу mystem по определению части речи для несуществующего/некорректного слова, а если предполагалось, что мы будем ориентироваться на правильные ответы, то будет странно, если мы будем пользоваться ответом в определении части речи. Также не вижу смысла ограничивать слова на вхождение в словарь по количеству встреч, т.к., как мы видели ранее, малое количество наоборот ухудшает качество, т.к. редкие слова в итоге превращаются в абсолютно другое, хотя должны были бы оставаться без изменений.

В рассмотренных методах при выборе исправления никак не использовался контекст. Про то как это можно сделать, можно почитать вот тут - https://habr.com/ru/post/346618/